In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats.mstats import winsorize
import datetime as dt

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

%matplotlib inline
# %matplotlib notebook

plt.rcParams["figure.figsize"] = (10,6)
# plt.rcParams['figure.dpi'] = 100

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.3f' % x)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Bidirectional,SimpleRNN
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


In [ ]:
pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df=pd.read_csv("kc_house_data.csv")

In [ ]:
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.000,3,1.000,1180,5650,1.000,0,0,...,7,1180,0,1955,0,98178,47.511,-122.257,1340,5650
1,6414100192,12/9/2014,538000.000,3,2.250,2570,7242,2.000,0,0,...,7,2170,400,1951,1991,98125,47.721,-122.319,1690,7639
2,5631500400,2/25/2015,180000.000,2,1.000,770,10000,1.000,0,0,...,6,770,0,1933,0,98028,47.738,-122.233,2720,8062
3,2487200875,12/9/2014,604000.000,4,3.000,1960,5000,1.000,0,0,...,7,1050,910,1965,0,98136,47.521,-122.393,1360,5000
4,1954400510,2/18/2015,510000.000,3,2.000,1680,8080,1.000,0,0,...,8,1680,0,1987,0,98074,47.617,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.000,3,2.500,1530,1131,3.000,0,0,...,8,1530,0,2009,0,98103,47.699,-122.346,1530,1509
21593,6600060120,2/23/2015,400000.000,4,2.500,2310,5813,2.000,0,0,...,8,2310,0,2014,0,98146,47.511,-122.362,1830,7200
21594,1523300141,6/23/2014,402101.000,2,0.750,1020,1350,2.000,0,0,...,7,1020,0,2009,0,98144,47.594,-122.299,1020,2007
21595,291310100,1/16/2015,400000.000,3,2.500,1600,2388,2.000,0,0,...,8,1600,0,2004,0,98027,47.535,-122.069,1410,1287


In [ ]:
df=df.drop("id",axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X=df[["feature1","feature2"]]
y=df["price"]

In [ ]:
X

,feature1,feature2
0,999.788,999.766
1,998.862,1001.042
2,1000.070,998.844
3,999.952,1000.441
4,1000.446,1000.339
...,...,...
995,1000.019,999.673
996,998.855,1000.020
997,1001.452,998.848
998,1000.771,998.563


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler=MinMaxScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Activation

In [ ]:
model=Sequential()
model.add(Dense(units=4,activation="relu",input_dim=X_train.shape[1]))
model.add(Dense(units=4,activation="relu"))
model.add(Dense(units=4,activation="relu"))
model.add(Dense(units=1))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 12        
                                                                 
 dense_1 (Dense)             (None, 4)                 20        
                                                                 
 dense_2 (Dense)             (None, 4)                 20        
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer="rmsprop",loss="mse")

In [ ]:
model.fit(x=X_train,y=y_train,batch_size=32,epochs=250)

Epoch 1/250
22/22 [==============================] - 3s 3ms/step - loss: 256693.9219
Epoch 2/250
22/22 [==============================] - 0s 3ms/step - loss: 256632.6562
Epoch 3/250
22/22 [==============================] - 0s 3ms/step - loss: 256571.5938
Epoch 4/250
22/22 [==============================] - 0s 3ms/step - loss: 256507.2656
Epoch 5/250
22/22 [==============================] - 0s 3ms/step - loss: 256439.0469
Epoch 6/250
22/22 [==============================] - 0s 3ms/step - loss: 256366.3594
Epoch 7/250
22/22 [==============================] - 0s 3ms/step - loss: 256288.4844
Epoch 8/250
22/22 [==============================] - 0s 3ms/step - loss: 256205.2344
Epoch 9/250
22/22 [==============================] - 0s 3ms/step - loss: 256116.2969
Epoch 10/250
22/22 [==============================] - 0s 3ms/step - loss: 256020.8750
Epoch 11/250
22/22 [==============================] - 0s 3ms/step - loss: 255918.9531
Epoch 12/250
22/22 [==============================] - 0s 3ms/st

In [ ]:
model.summary(
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 12        
                                                                 
 dense_1 (Dense)             (None, 4)                 20        
                                                                 
 dense_2 (Dense)             (None, 4)                 20        
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.history.history

{'loss': [256693.921875,
  256632.65625,
  256571.59375,
  256507.265625,
  256439.046875,
  256366.359375,
  256288.484375,
  256205.234375,
  256116.296875,
  256020.875,
  255918.953125,
  255809.640625,
  255693.234375,
  255568.796875,
  255435.75,
  255294.0625,
  255142.828125,
  254981.9375,
  254810.90625,
  254628.890625,
  254436.1875,
  254232.640625,
  254015.015625,
  253785.8125,
  253542.359375,
  253285.53125,
  253013.671875,
  252727.59375,
  252426.1875,
  252108.28125,
  251775.34375,
  251426.328125,
  251056.578125,
  250666.484375,
  250258.828125,
  249833.515625,
  249386.875,
  248918.53125,
  248427.546875,
  247917.515625,
  247382.3125,
  246824.484375,
  246240.984375,
  245632.359375,
  245000.609375,
  244341.046875,
  243654.921875,
  242938.109375,
  242196.609375,
  241427.296875,
  240629.328125,
  239793.9375,
  238935.453125,
  238043.078125,
  237117.0,
  236156.1875,
  235161.78125,
  234129.6875,
  233072.484375,
  231975.078125,
  230839.26562

In [ ]:
pd.DataFrame(model.history.history)

,loss
0,256693.922
1,256632.656
2,256571.594
3,256507.266
4,256439.047
...,...
245,538.946
246,527.802
247,517.649
248,506.758


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
model.evaluate(X_train,y_train,verbose=0)

488.47039794921875

In [ ]:
model.evaluate(X_test,y_test,verbose=0)

477.3296813964844

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
my_dict={"Act":y_test,"Pred":y_pred.reshape(-1,)}
compare=pd.DataFrame(my_dict)
compare.sample(10)

,Act,Pred
542,689.173,644.420
522,456.900,466.901
904,627.640,602.451
198,418.457,438.043
922,425.483,450.468
82,462.005,475.792
948,433.423,455.528
290,448.632,462.671
947,412.907,437.115
901,503.216,508.434


In [ ]:
def ev(act,pred):
  mae=mean_absolute_error(act,pred)
  mse=mean_squared_error(act,pred)
  rmse=np.sqrt(mean_squared_error(act,pred))
  score=r2_score(act,pred)
  return print("r2_score:",score,"\nmae:",mae,"\nmse:",mse,"\nrmse:",rmse)

In [ ]:
ev(y_test,y_pred)

r2_score: 0.9457088233003602 
mae: 17.542972486431246 
mse: 477.3297446957483 
rmse: 21.84787734988798


In [ ]:
scaler=MinMaxScaler().fit(X)
import pickle
pickle.dump(scaler, open("scaler_fake_reg","wb"))

In [ ]:
X_scaled=scaler.transform(X)

In [ ]:
final=Sequential()
final.add(Dense(units=4,activation="relu"))
final.add(Dense(units=4,activation="relu"))
final.add(Dense(units=4,activation="relu"))
final.add(Dense(units=1))
final.compile(optimizer="rmsprop",loss="mse")
final.fit(x=X_scaled,y=y,batch_size=32,epochs=250)


Epoch 1/250
32/32 [==============================] - 1s 3ms/step - loss: 257433.7188
Epoch 2/250
32/32 [==============================] - 0s 3ms/step - loss: 257368.0625
Epoch 3/250
32/32 [==============================] - 0s 3ms/step - loss: 257309.2188
Epoch 4/250
32/32 [==============================] - 0s 3ms/step - loss: 257241.5469
Epoch 5/250
32/32 [==============================] - 0s 3ms/step - loss: 257167.5938
Epoch 6/250
32/32 [==============================] - 0s 3ms/step - loss: 257087.1719
Epoch 7/250
32/32 [==============================] - 0s 3ms/step - loss: 256997.7344
Epoch 8/250
32/32 [==============================] - 0s 3ms/step - loss: 256898.1719
Epoch 9/250
32/32 [==============================] - 0s 3ms/step - loss: 256785.2031
Epoch 10/250
32/32 [==============================] - 0s 3ms/step - loss: 256653.3125
Epoch 11/250
32/32 [==============================] - 0s 3ms/step - loss: 256500.5625
Epoch 12/250
32/32 [==============================] - 0s 3ms/st

In [ ]:
final.save("model_fake_reg.h5")

In [ ]:
from tensorflow.keras.models import load_model
model_fake_reg=load_model("model_fake_reg.h5")
scaler_fake_reg=pickle.load(open("scaler_fake_reg","rb"))

In [ ]:
scaler_fake_reg=pickle.load(open("scaler_fake_reg","rb"))